In [18]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import torch
from torch import nn
from pytorch_transformers import BertForSequenceClassification, BertTokenizer

from fastai.callbacks import Learner, LRFinder
from fastai.text import BaseTokenizer, List, Tokenizer, Vocab, TextDataBunch, TextClasDataBunch, partial, pad_collate

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
train, val = train_test_split(train)

In [3]:
print(train.shape, val.shape)
train.head()

(119678, 8) (39893, 8)


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
10944,1cea567139a0eed2,"Don't worry, there is a backlog and the next s...",0,0,0,0,0,0
78980,d359010b09301277,Cheerleader \n\nYour a dumb fuck for having an...,1,0,1,0,1,0
80001,d61467806657cf0e,I thought it was a term that pretty explicitly...,0,0,0,0,0,0
135025,d23cef3a79955a9f,ok your site already fucked so i cant fuck it ...,1,0,1,0,1,1
156483,ced23980bd164a3d,To be completely honest that looks 10x as wors...,0,0,0,0,0,0


In [4]:
bert_tok = BertTokenizer.from_pretrained("bert-base-uncased")

class FastAiBertTokenizer(BaseTokenizer):
    def __init__(self, tokenizer:BertTokenizer, max_seq_len:int=128, **kwargs):
        self._pretrained_tokenizer = tokenizer
        self.max_seq_len = max_seq_len

    def __call__(self, *args, **kwargs):
        return self

    def tokenizer(self, t:str) -> List[str]:
        return ["[CLS]"] + self._pretrained_tokenizer.tokenize(t)[:self.max_seq_len - 2] + ["[SEP]"]
    
fastai_tokenizer = Tokenizer(
    tok_func=FastAiBertTokenizer(bert_tok, max_seq_len=128), 
    pre_rules=[], 
    post_rules=[]
)

fastai_bert_vocab = Vocab(list(bert_tok.vocab.keys()))

In [19]:
databunch = TextDataBunch.from_df(
    ".", 
    train, 
    val, 
    test,
    tokenizer=fastai_tokenizer,
    vocab=fastai_bert_vocab,
    include_bos=False,
    include_eos=False,
    text_cols="comment_text",
    label_cols=list(train.columns[2:]),
    bs=4,
    collate_fn=partial(pad_collate, pad_first=False, pad_idx=0)
)

In [20]:
loss_func = nn.BCEWithLogitsLoss()

bert_model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=6)

learner = Learner(databunch, bert_model, loss_func=loss_func)
learner = learner.to_fp16()

In [21]:
learner.lr_find()

LR Finder is complete, type {learner_name}.recorder.plot() to see the graph.


AttributeError: 'list' object has no attribute 'size'

In [22]:
# learner.recorder.plot()

In [10]:
# learner.fit_one_cycle(1, max_lr=5e-5)